# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686578


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:35,  3.40s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:05,  2.42s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:45,  1.74s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:32,  1.29s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:09<00:00,  8.60it/s]

Rendering models:  90%|█████████ | 27/30 [00:09<00:00,  8.27it/s]

Rendering models:  93%|█████████▎| 28/30 [00:09<00:00,  6.97it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  7.06it/s]

not-logged-in-68d3d82149e88f995533     0.448375
smithkurtis                            0.000062
shirogane                              0.000080
not-logged-in-a9247261537fa1da33fe    37.222315
mkehrli                                0.000086
not-logged-in-3e831daf31432a82f496     0.303667
mtajnai                                0.000091
justsomenoodles                        0.000076
jmfranci                               0.000166
not-logged-in-39f226bd4e0a8ba276a6     0.001056
not-logged-in-bf953ebf72a01f7ce112     0.004305
not-logged-in-5f30b2bb9b7f9d49da41     0.157281
crush202020                            0.000390
jmartinez468                           0.006124
UnbiasedBrigade                        0.000698
AlexYoung35                            0.000097
databanana                             0.000064
AshleyNordlund                         0.000216
Lavadude                               0.001977
TateNachampassak                       0.000212
not-logged-in-0582d7599288fe425356     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())